In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/instquality/")

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/instquality/")

import warnings
warnings.filterwarnings("ignore")

Mounted at /content/drive


In [7]:
import pandas as pd
import numpy as np

from functions.spec import run_unified_regularized_regression, filter_cols
from functions.utils import check_split_year
from functions.summary import get_benchmark_stats

### Incremental signal regressions

In [8]:
# load data
df = pd.read_pickle("saved/df.dat")

# define columns and inputs
id_cols = ["country","year","iso_code_1","iso_code_2","region"]
exclude = id_cols + [c for c in df.columns if c.startswith("tgt_")]
macro_cols = filter_cols([c for c in df.columns if c.startswith("wb_") and not c.startswith("wb_iq_") and c not in exclude], [])
iq_cols = filter_cols([c for c in df.columns if (c.startswith("wb_iq_") or (not c.startswith("wb_") and c not in exclude))], [])
macro_cols += ["tgt_spread"]
y_name = "tgt_spread_t1"

# define models to run
models = ['lasso', 'ridge', 'elastic']

# iterate over split shares
split_shares = [0.75, 0.8, 0.85]
for split_share in split_shares:

    # get split year and benchmarks
    split_year = check_split_year(df, y_name, split_share)
    if split_share != 0.75:
        name_prefix = f"signal_t1_{split_share*100:.0f}"
    else:
        name_prefix = "signal_t1"
    print("\n" + "=" * 100)
    print(f"Runnning {split_share*100:.0f}-{(1-split_share)*100:.0f} split...")
    print(f"Split year for {split_share*100:.0f}-{(1-split_share)*100:.0f} split: {split_year}")
    benchmark_df = get_benchmark_stats(df, split_share=split_share)
    h_benchmark = benchmark_df[benchmark_df["horizon"] == 1].iloc[0]
    r2_benchmark = h_benchmark["R2_test"]
    rmse_benchmark = h_benchmark["RMSE_test"]

    # run spec for models
    run_unified_regularized_regression(
        name_prefix=name_prefix,
        data_path="saved/df.dat",
        macro_cols=macro_cols,
        iq_cols=iq_cols,
        y_name=y_name,
        spread_col="tgt_spread",
        year_col="year",
        split_year=split_year,
        model_types=models,
        agnostic=False,
        param_factors=np.logspace(-3, 10, 25),
        l1_ratios=[0.25, 0.5, 0.75],
        use_clustering=True,
        corr_threshold=0.9,
        use_expanding_cv=True,
        min_train_years=8,
        cv_stride=2,
        r2_benchmark=r2_benchmark,
        rmse_benchmark=rmse_benchmark,
        early_stop_threshold=0.10,
        stability_iterations=100,
        save_results=True,
        output_dir="specs",
        n_jobs=-1,
        parallel_param_factors=True,
        compute_shap=True,
        max_shap_samples=500,
        force_interpretability=False,
        r2_convergence_threshold=1e-4,
        r2_convergence_count=3,
        multicollinearity_check=True,
        perform_interpretability=True
    )


Runnning 75-25 split...
Split year for 75-25 split: 2014

UNIFIED REGULARIZED REGRESSION: SIGNAL_T1
Specification: β=1 (fixed at mean reversion)
Models: LASSO, RIDGE, ELASTIC
Elastic Net L1 ratios: [0.25, 0.5, 0.75]
Feature clustering: ON (threshold=0.9)
Statistical inference: SHAP values, Stability Selection (Lasso/Elastic Net only)
R² convergence: Stop after 3 consecutive changes < 0.0001

Train: 1960-2014 (1564 obs)
Test:  2015-2023 (534 obs)

β=1 specification active
After removing NaN: 1564 train, 534 test obs

Benchmark metrics:
  R² (test): 0.9074
  RMSE (test): 1.2728

MULTICOLLINEARITY DIAGNOSTICS (Core Variables Only)

Found 56 core variable pairs with correlation above 0.9:



,core_variable_1,core_variable_2,correlation
0,p5d_xrreg,p5d_xconst,0.999259
1,wb_fp_cpi_totl_zg,wb_ny_gdp_defl_kd_zg,0.999178
2,p5d_democ,p5d_xconst,0.999040
3,p5d_democ,p5d_xrreg,0.997578
4,p5d_autoc,p5d_xrreg,0.997499
5,p5d_autoc,p5d_xconst,0.994880
6,p5d_democ,p5d_autoc,0.991070
7,fiw_b,fiw_pr,0.987332
8,fiw_f,fiw_cl,0.975276
9,fiw_a,fiw_pr,0.971855



Note: These correlations are reported for transparency.
Regularization handles multicollinearity. Interpretation focuses on stable patterns.

Clustering variants of same variables...
  Original features: 972
  After clustering: 640
  Number of clusters: 640

Features used in models: 640


--------------------------------------------------------------------------------
RUNNING LASSO
--------------------------------------------------------------------------------

Selecting optimal parameter via expanding window CV...
Optimal parameter: 0.483293

Stopping: model has 0 active variables at factor 6.189658188912603



,factor,param,R²_train,R²_test,RMSE_test,Active_vars,DM_stat,DM_p
0,0.001000,0.000483,0.952514,0.489696,2.988004,565,-11.909514,0.000000e+00
1,0.003481,0.001682,0.945931,0.531183,2.863972,473,-12.331897,0.000000e+00
2,0.012115,0.005855,0.932586,0.614423,2.597301,351,-11.398731,0.000000e+00
3,0.042170,0.020380,0.910313,0.823581,1.756868,204,-5.101179,3.375450e-07
4,0.146780,0.070938,0.864961,0.879063,1.454611,90,-2.980710,2.875811e-03
5,0.510897,0.246913,0.816900,0.902750,1.304405,13,-1.227936,2.194709e-01
6,1.778279,0.859430,0.758771,0.908984,1.261905,1,0.997312,3.186132e-01



Best LASSO:
  Parameter: 0.859430
  R² (test): 0.9090
  ΔR² vs benchmark: +0.16 p.p.
  RMSE (test): 1.2619
  ΔRMSE vs benchmark: -0.0109
  Active vars: 1
  Active variables: wb_fp_cpi_totl_zg
  DM test: stat=1.00, p=0.319

Skipping interpretability analysis: no significant incremental value over mean reversion benchmark (DM p=0.319)

Interpretation: Macro/IQ variables do not provide statistically significant
incremental predictive power beyond the mean reversion benchmark.


--------------------------------------------------------------------------------
RUNNING RIDGE
--------------------------------------------------------------------------------

Selecting optimal parameter via expanding window CV...
Optimal parameter: 24297.810658

Early stopping: R² converged after 3 iterations.



...


Best ELASTIC (L1=0.25):
  Parameter: 2.887529
  R² (test): 0.8928
  ΔR² vs benchmark: -0.11 p.p.
  RMSE (test): 1.3264
  ΔRMSE vs benchmark: +0.0068
  Active vars: 9
  Active variables: wb_fp_cpi_totl_zg_ma3, wb_fp_cpi_totl_zg, wb_fp_cpi_totl_zg_ma10, wb_ny_gdp_defl_kd_zg, wb_ny_gdp_defl_kd_zg_delta3, wb_ny_gdp_defl_kd_zg_delta, wb_ny_gdp_defl_kd_zg_ma3, wb_ny_gdp_defl_kd_zg_ma5, wb_ny_gdp_defl_kd_zg_ma10
  DM test: stat=-0.54, p=0.587

Skipping interpretability analysis: no significant incremental value over mean reversion benchmark (DM p=0.587)

Interpretation: Macro/IQ variables do not provide statistically significant
incremental predictive power beyond the mean reversion benchmark.


--------------------------------------------------------------------------------
RUNNING ELASTIC (L1=0.5)
--------------------------------------------------------------------------------

Selecting optimal parameter via expanding window CV...
Optimal parameter: 0.885867

Stopping: model has 0 active 

,factor,param,R²_train,R²_test,RMSE_test,Active_vars,DM_stat,DM_p
0,0.001000,0.000886,0.945074,0.806593,1.781718,561,-4.592342,0.000004
1,0.003481,0.003083,0.937858,0.829736,1.671722,477,-4.104059,0.000041
2,0.012115,0.010733,0.925940,0.839077,1.625221,365,-3.843567,0.000121
3,0.042170,0.037357,0.902887,0.861829,1.505955,228,-2.563701,0.010356
4,0.146780,0.130027,0.860400,0.877566,1.417603,88,-2.164183,0.030450
5,0.510897,0.452587,0.821048,0.889496,1.346764,16,-1.110554,0.266760
6,1.778279,1.575319,0.757064,0.893026,1.325078,7,-0.417115,0.676594



Best ELASTIC (L1=0.5):
  Parameter: 1.575319
  R² (test): 0.8930
  ΔR² vs benchmark: -0.09 p.p.
  RMSE (test): 1.3251
  ΔRMSE vs benchmark: +0.0055
  Active vars: 7
  Active variables: wb_fp_cpi_totl_zg_ma3, wb_fp_cpi_totl_zg, wb_fp_cpi_totl_zg_ma10, wb_ny_gdp_defl_kd_zg, wb_ny_gdp_defl_kd_zg_delta3, wb_ny_gdp_defl_kd_zg_delta, wb_ny_gdp_defl_kd_zg_ma3
  DM test: stat=-0.42, p=0.677

Skipping interpretability analysis: no significant incremental value over mean reversion benchmark (DM p=0.677)

Interpretation: Macro/IQ variables do not provide statistically significant
incremental predictive power beyond the mean reversion benchmark.


--------------------------------------------------------------------------------
RUNNING ELASTIC (L1=0.75)
--------------------------------------------------------------------------------

Selecting optimal parameter via expanding window CV...
Optimal parameter: 0.483293

Stopping: model has 0 active variables at factor 21.54434690031882



,factor,param,R²_train,R²_test,RMSE_test,Active_vars,DM_stat,DM_p
0,0.001000,0.000483,0.946669,0.796456,1.827813,578,-4.830032,0.000001
1,0.003481,0.001682,0.940853,0.822328,1.707702,497,-4.338886,0.000014
2,0.012115,0.005855,0.929421,0.838173,1.629779,378,-3.871751,0.000108
3,0.042170,0.020380,0.910026,0.856380,1.535362,246,-2.849457,0.004379
4,0.146780,0.070938,0.871310,0.874852,1.433229,103,-2.131835,0.033020
5,0.510897,0.246913,0.829961,0.886429,1.365328,18,-1.462753,0.143535
6,1.778279,0.859430,0.782267,0.893282,1.323494,4,-0.264940,0.791055
7,6.189658,2.991419,0.571927,0.891916,1.331937,2,-1.124226,0.260917



Best ELASTIC (L1=0.75):
  Parameter: 0.859430
  R² (test): 0.8933
  ΔR² vs benchmark: -0.06 p.p.
  RMSE (test): 1.3235
  ΔRMSE vs benchmark: +0.0039
  Active vars: 4
  Active variables: wb_fp_cpi_totl_zg_ma3, wb_fp_cpi_totl_zg, wb_fp_cpi_totl_zg_ma10, wb_ny_gdp_defl_kd_zg
  DM test: stat=-0.26, p=0.791

Skipping interpretability analysis: no significant incremental value over mean reversion benchmark (DM p=0.791)

Interpretation: Macro/IQ variables do not provide statistically significant
incremental predictive power beyond the mean reversion benchmark.


COMPLETED IN 270.3s



### Priced-in level regressions

In [ ]:
# load data
df = pd.read_pickle("saved/df.dat")

# define columns
id_cols = ["country", "year", "iso_code_1", "iso_code_2", "region"]
exclude = id_cols + [c for c in df.columns if c.startswith("tgt_")]
macro_cols = filter_cols([c for c in df.columns if c.startswith("wb_") and not c.startswith("wb_iq_") and c not in exclude], [])
iq_cols = filter_cols([c for c in df.columns if (c.startswith("wb_iq_") or (not c.startswith("wb_") and c not in exclude))], [])
y_name = "tgt_spread"

# define models to run
models = ['lasso', 'ridge', 'elastic']

# iterate over split shares
split_shares = [0.75, 0.8, 0.85]
for split_share in split_shares:

    # get split year
    split_year = check_split_year(df, y_name, split_share)
    if split_share != 0.75:
        name_prefix = f"levels_t0_{split_share*100:.0f}"
    else:
        name_prefix = "levels_t0"
    print("\n" + "=" * 100)
    print(f"Runnning {split_share*100:.0f}-{(1-split_share)*100:.0f} split...")
    print(f"Split year for {split_share*100:.0f}-{(1-split_share)*100:.0f} split: {split_year}")

    # run spec for models
    results = run_unified_regularized_regression(
        name_prefix=name_prefix,
        data_path="saved/df.dat",
        macro_cols=macro_cols,
        iq_cols=iq_cols,
        use_clustering=True,
        corr_threshold=0.9,
        y_name=y_name,
        spread_col="tgt_spread",
        year_col="year",
        split_year=split_year,
        model_types=models,
        agnostic=True,
        param_factors=np.logspace(-2, 5, 15),
        l1_ratios=[0.25, 0.5, 0.75],
        use_expanding_cv=True,
        min_train_years=8,
        cv_stride=2,
        r2_benchmark=None,
        rmse_benchmark=None,
        early_stop_threshold=0.10,
        save_results=True,
        output_dir="specs",
        n_jobs=-1
    )

Runnning 75-25 split...
Split year for 75-25 split: 2015

UNIFIED REGULARIZED REGRESSION: LEVELS_T0
Specification: Agnostic (β unrestricted)
Models: LASSO, RIDGE, ELASTIC
Elastic Net L1 ratios: [0.25, 0.5, 0.75]
Feature clustering: ON (threshold=0.9)
Statistical inference: SHAP values, Stability Selection (Lasso/Elastic Net only)
R² convergence: Stop after 3 consecutive changes < 0.0001

Train: 1960-2015 (1652 obs)
Test:  2016-2024 (541 obs)

MULTICOLLINEARITY DIAGNOSTICS (Core Variables Only)

Found 60 core variable pairs with correlation above 0.9:



,core_variable_1,core_variable_2,correlation
0,p5d_democ,p5d_xconst,0.999063
1,wb_fp_cpi_totl_zg,wb_ny_gdp_defl_kd_zg,0.999056
2,p5d_xrreg,p5d_xconst,0.998977
3,p5d_democ,p5d_xrreg,0.997217
4,p5d_autoc,p5d_xrreg,0.997004
5,p5d_autoc,p5d_xconst,0.993404
6,p5d_democ,p5d_autoc,0.989513
7,fiw_b,fiw_pr,0.988235
8,fiw_cl,fiw_f,0.976274
9,fiw_a,fiw_pr,0.974487



Note: These correlations are reported for transparency.
Regularization handles multicollinearity. Interpretation focuses on stable patterns.

Clustering variants of same variables...
  Original features: 972
  After clustering: 634
  Number of clusters: 634

Features used in models: 634


--------------------------------------------------------------------------------
RUNNING LASSO
--------------------------------------------------------------------------------

Selecting optimal parameter via expanding window CV...
Optimal parameter: 0.003793

Stopping: model has 0 active variables at factor 3162.2776601683795



,factor,param,R²_train,R²_test,RMSE_test,Active_vars,DM_stat,DM_p
0,0.010000,0.000038,0.963680,-0.100415,4.396000,632,NaN,NaN
1,0.031623,0.000120,0.963665,-0.084150,4.363390,629,NaN,NaN
2,0.100000,0.000379,0.963350,-0.049959,4.294035,605,NaN,NaN
3,0.316228,0.001199,0.961327,-0.009447,4.210379,542,NaN,NaN
4,1.000000,0.003793,0.954384,0.178697,3.797790,446,NaN,NaN
5,3.162278,0.011994,0.939810,0.432268,3.157555,329,NaN,NaN
6,10.000000,0.037927,0.915739,0.472107,3.044755,203,NaN,NaN
7,31.622777,0.119935,0.862338,0.495854,2.975484,86,NaN,NaN
8,100.000000,0.379269,0.790401,0.514568,2.919736,24,NaN,NaN
9,316.227766,1.199354,0.680979,0.303349,3.497735,7,NaN,NaN



Best LASSO:
  Parameter: 0.379269
  R² (test): 0.5146
  RMSE (test): 2.9197
  Active vars: 24


INTERPRETABILITY ANALYSIS: WHICH VARIABLES ARE PRICED IN?
(LASSO)

METHOD 1: SHAP VALUES (Feature Attribution)

Computing SHAP values...

Top 19 core variables by SHAP importance:



...


Best ELASTIC (L1=0.75):
  Parameter: 0.119935
  R² (test): 0.6289
  RMSE (test): 2.4831
  Active vars: 134


INTERPRETABILITY ANALYSIS: WHICH VARIABLES ARE PRICED IN?
(ELASTIC (L1=0.75))

METHOD 1: SHAP VALUES (Feature Attribution)

Computing SHAP values...

Top 20 core variables by SHAP importance:



,core_variable,variants,shap_importance,shap_importance_pct,n_variants
74,ief_monetary_freedom,"[ief_monetary_freedom_ma3, ief_monetary_freedo...",0.810362,9.740875,7
19,efw_3b_standard_deviation_of_inflation,"[efw_3b_standard_deviation_of_inflation_t-1, e...",0.511914,6.153407,8
28,efw_4bii_costs_of_importing_and_exporting,[efw_4bii_costs_of_importing_and_exporting_t-5...,0.429920,5.167810,6
2,efw_1b_transfers_and_subsidies,"[efw_1b_transfers_and_subsidies_t-3, efw_1b_tr...",0.410927,4.939512,6
87,wb_bn_cab_xoka_gd_zs,"[wb_bn_cab_xoka_gd_zs_ma5, wb_bn_cab_xoka_gd_z...",0.406334,4.884294,7
51,efw_5ciii_impartial_public_administration,[efw_5ciii_impartial_public_administration_ma1...,0.309737,3.723162,4
13,efw_2e_legal_integrity,"[efw_2e_legal_integrity_ma10, efw_2e_legal_int...",0.279650,3.361505,5
59,fiw_a,"[fiw_a_ma10, fiw_a_t-5, fiw_a_t-3, fiw_a_delta...",0.245268,2.948216,5
38,efw_5aii_private_sector_credit,"[efw_5aii_private_sector_credit_t-3, efw_5aii_...",0.245131,2.946569,8
31,efw_4di_financial_openness,"[efw_4di_financial_openness_t-5, efw_4di_finan...",0.243015,2.921132,7



METHOD 2: STABILITY SELECTION

Running stability selection (100 iterations)...

Core variables with stable variants (aggregated from 67 stable features):



,core_variable,variants,selection_frequency,mean_coef,std_coef,n_variants
6,efw_2f_contracts,[efw_2f_contracts_delta3],1.00,-0.114977,0.038117,1
5,efw_2e_legal_integrity,[efw_2e_legal_integrity_ma10],1.00,-0.360907,0.098910,1
12,efw_4ai_trade_tax_revenue,[efw_4ai_trade_tax_revenue_t-3],1.00,-0.172761,0.056832,1
15,efw_4bii_costs_of_importing_and_exporting,[efw_4bii_costs_of_importing_and_exporting_t-5],1.00,-0.301060,0.049379,1
24,efw_5ciii_impartial_public_administration,[efw_5ciii_impartial_public_administration_ma10],1.00,-0.348119,0.084065,1
26,efw_gender_disparity_index,[efw_gender_disparity_index_ma10],1.00,0.287805,0.073636,1
22,efw_5ci_regulatory_burden,[efw_5ci_regulatory_burden_t-3],1.00,0.133594,0.045129,1
41,wb_ny_gdp_minr_rt_zs,[wb_ny_gdp_minr_rt_zs_ma10],1.00,0.253204,0.040108,1
32,p5d_autoc,[p5d_autoc_ma10],1.00,-0.126239,0.042886,1
33,p5d_durable,[p5d_durable],1.00,-0.194405,0.041324,1



COMPLETED IN 587.3s

